In [ ]:
import pandas as pd
import json
import random
import hashlib
import jsonlines
from pathlib import Path

In [ ]:
def str2hashkey(s, length=16):
    return hashlib.md5(s.encode()).hexdigest()[:length]

## CrossTask

In [ ]:
samples = []
with open("/data/CrossTask/crosstask_release/tasks_primary.txt") as f:
    lines = f.readlines()
    print(len(lines) // 6)
raw_text = "".join(lines)
sample_texts = raw_text.split("\n\n")
for sample_text in sample_texts:
    lines = sample_text.split("\n")
    if len(lines) > 2:
        try:
            samples.append({
                "id": lines[0].strip(),
                "task": lines[1].strip(),
                "url": lines[2].strip(),
                "steps": [x.strip() for x in lines[4].split(",")],
                "type": "primary"
            })
        except:
            print(lines)
            raise
print(len(samples))
with open("/data/CrossTask/crosstask_release/tasks_related.txt") as f:
    lines = f.readlines()
    print(len(lines) // 6)
raw_text = "".join(lines)
sample_texts = raw_text.split("\n\n")
for sample_text in sample_texts:
    lines = sample_text.split("\n")
    if len(lines) > 2:
        try:
            samples.append({
                "id": lines[0].strip(),
                "task": lines[1].strip(),
                "url": lines[2].strip(),
                "steps": [x.strip() for x in lines[4].split(",")],
                "type": "related"
            })
        except:
            print(lines)
            raise
print(len(samples))
with jsonlines.open("/data/CrossTask/crosstask_release/tasks.jsonl", "w") as writer:
    writer.write_all(samples)

In [ ]:
cross_vtt = []
with jsonlines.open("/data/reason/CrossTask/crosstask_release/tasks.jsonl") as reader:
    tasks = list(reader)
tasks = {task["id"]:task for task in tasks}
with open("/data/reason/CrossTask/crosstask_release/videos_val.csv") as f:
    val_samples = f.readlines()
val_videos = set([x.split(",")[1] for x in val_samples])
for path in Path("/data/reason/CrossTask/crosstask_release/annotations").glob("*.csv"):
    try:
        splits = path.stem.split("_")
        task_id = splits[0]
        youtube_id = "_".join(splits[1:])
        sample_id = str2hashkey(path.name)
        with path.open() as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]

        cross_vtt.append({
            "id": sample_id,
            "youtube_id": youtube_id,
            "ori": "cross",
            "split": "test" if youtube_id in val_videos else "train",
            "duration": -1,
            "annotation": [
                {
                    "clip_id": f"{sample_id}_{len(lines)}_{i}",
                    "segment": [float(x) for x in line.split(",")[1:3]],
                    "label": tasks[task_id]["steps"][int(line.split(",")[0])-1]
                } for i, line in enumerate(lines)
            ]
        })
    except:
        print(path)
        print(tasks[task_id])
        raise

In [ ]:
len(cross_vtt)

## VAR (ActivityNet)

In [ ]:
with open("/data/reason/ActivityNet/var_val_v1.0.json") as f:
    data = json.load(f)

In [ ]:
key = random.choice(list(data.keys()))
print(key)
print(data[key].keys())
print(json.dumps(data[key], indent=2))

In [ ]:
str2hashkey(json.dumps(data[key], indent=2), length=16)

In [ ]:
def var2vtt(data):
    samples = []
    keys = set()
    for item in data:
        sample_id = str2hashkey(json.dumps(item["events"], indent=2), length=16)
        if sample_id not in keys:
            keys.add(sample_id)
            samples.append({
                "id": sample_id,
                "youtube_id": item["events"][0]["video_id"],
                "ori": "var",
                "split": item["split"],
                "duration": item["events"][0]["duration"],
                "annotation": [
                    {
                        "clip_id": f"{sample_id}_{len(item['events'])}_{i}",
                        "segment": event["timestamp"],
                        "label": event["sentence"]
                    } for i, event in enumerate(item["events"])
                ]
            })
    return samples

In [ ]:
with open("/data/reason/ActivityNet/var_train_v1.0.json") as f:
    train = json.load(f)
with open("/data/reason/ActivityNet/var_val_v1.0.json") as f:
    val = json.load(f)
with open("/data/reason/ActivityNet/var_test_v1.0.json") as f:
    test = json.load(f)
print(len(train)+len(val)+len(test))

In [ ]:
var_vtt = var2vtt(list(train.values()) + list(val.values()) + list(test.values()))
print(len(var_vtt))

## COIN

In [ ]:
with jsonlines.open("../../Real_TVR/data/videos.jsonl") as f:
    coin = list(f)
print(len(coin))

In [ ]:
random.choice(coin)

In [ ]:
"training".replace("ing", "")

In [ ]:
def coin2vtt(data):
    samples = []
    # keys = {}
    for item in data:
        sample_id = str2hashkey(json.dumps(item, indent=2), length=16)
        # if sample_id not in keys:
            # keys[sample_id] = item
        samples.append({
            "id": sample_id,
            "youtube_id": item["id"],
            "ori": "coin",
            "split": item["subset"].replace("ing", ""),
            "duration": item["duration"],
            "annotation": [
                {
                    "clip_id": f"{sample_id}_{len(item['annotation'])}_{i}",
                    "segment": step["segment"],
                    "label": step["label"]
                } for i, step in enumerate(item["annotation"])
            ]
        })
        # else:
        #     print(item)
        #     print(keys[sample_id])
        #     print()
    return samples

In [ ]:
coin_vtt = coin2vtt(coin)
print(len(coin_vtt))

In [ ]:
## Health Samples

In [ ]:
with 